<a href="https://colab.research.google.com/github/tareqdandachi/QNN-DiffEq.jl-Tutorial/blob/main/QNN_DiffEq_jl_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### QNN Tutorial
# Solving Differential Equations Efficiently in Julia using `DifferentialEquations.jl`

In this tutorial, we will be simulating an RCSJ model of a superconducting Josephson Junction using Julia and applying different levels of optimization.

If you have your an already intalled interactive notebook with Julia, feel free to use it and skip to the Tutorial section (you might want to check that you have all the packages listed in the setup section's intro). Otherwise, if you are using Google Colab, follow the "Setting Up Julia On Colab" section.

# Notes on indexing

Julia uses one indexing

<img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="50" />

#  _Setting Up Julia On Colab_

To follow with this tutorial, you need to use a julia kernel. If you are using Google Colab, which defaults to python, follow the setup instructions below which should take \<15 mins to setup julia.

We will be using the following packages for this tutorial:
- BenchmarkTools
- Plots
- DifferentialEquations
- LinearAlgebra
- StaticKernels
- LoopVectorization

### Step 1: Run this following cell

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.7.1" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools Plots DifferentialEquations LinearAlgebra StaticKernels LoopVectorization"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
fi

Installing Julia 1.7.1 on the current Colab Runtime...
2022-07-07 18:56:44 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.7/julia-1.7.1-linux-x86_64.tar.gz [123374573/123374573] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing Julia package Plots...
Installing Julia package DifferentialEquations...


### Step 2: Hit refresh on your browser

Refreshing the page will reinitialize your instance and connect to a Julia kernel instead of Python. If you want to use a GPU, this is where you would add GPU acceleration under `Runtime` > `Change Runtime Type`

### Step 3: Verify the julia installation works

In [1]:
versioninfo()

Julia Version 1.7.1
Commit ac5cc99908 (2021-12-22 19:35 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, broadwell)
Environment:
  JULIA_NUM_THREADS = 2


# Tutorial


We will be modelling a Josephson Junction using the RCSJ model to recreate a JJ Washboard Potential. Our system can be described by this second order differential equation:

\

$$\dfrac{\Phi_0}{2\pi}C\ddot\phi+\dfrac{\Phi_0}{2\pi R} \dot\phi+I_c \sin\phi=\dfrac{\Phi_0}{2\pi R} I$$

\
Our system has a mechanical analog of a force driven pendulum:

\

$$ml^2\ddot\phi+\Gamma\dot\phi+mgl\sin\phi=\tau_{applied}$$

\

Where $ml^2\equiv\dfrac{\Phi_0}{2\pi}C$, $\;\;\Gamma\equiv\dfrac{\Phi_0}{2\pi R}$, $\;\; mg=I_c$, $\;\; l=1$ and $\tau_{\text{applied}} = F_{\text{drive}}\cdot l\equiv\dfrac{\Phi_0}{2\pi R} I$

\
*NOTE: this work is based on a MATLAB visualization made for 6.s079 here [tareqdandachi/RCSJ-Washboard-Potential](https://github.com/tareqdandachi/RCSJ-Washboard-Potential).*

### Importing packages

In [2]:
using BenchmarkTools
using Plots
using DifferentialEquations
using LinearAlgebra

### Start out with linearized near zero since phase driven to 0. Ideal JJ eq

$$v(t) = \dfrac{\Phi_0}{2\pi I_c}\dfrac{di}{dt} + iR_N$$

hmmm...

instead lets do RC circuit first

You have an RC circuit, equation is:

$$ RC \dfrac{dv_c}{dt} + v_c = v_{in}(t) $$

Simple scalar first-order diff eq.

let $f(v) = \dfrac{dv_c}{dt} = \dfrac{1}{RC} \left( v_{in}(t) - v_c \right)$

In [4]:
omega = 1

vin(t) = sin(2*pi*omega*t)

R = 100 #Ohms
C = 1e-9 # F

f(v, params, t) = 1/(R*C) * (vin(t) - v)

v0 = 1/2
tspan = (0.0,10.0)

prob = ODEProblem(f,v0,tspan)
sol = solve(prob, Tsit5(), reltol=1e-8, abstol=1e-8)

plot(sol,linewidth=5,title="Solution to the linear ODE with a thick line",
     xaxis="Time (t)",yaxis="u(t) (in μm)",label="My Thick Line!") # legend=false
plot!(sol.t, t->(exp(-t/(R*C)) + 1/sqrt(R^2 + (1/(omega * C))^2) * cos(omega*t + atan(1/(omega*R*C)))), lw=3,ls=:dash,label="True Solution!")

SyntaxError: ignored